In [ ]:
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

from nemo.siren import Siren
from nemo.plotting import plot_3d_points, plot_surface, plot_path_3d

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# autoreload
%load_ext autoreload
%autoreload 2

## AirSim Landscape Mountains

In [ ]:
vertices = np.load('../data/landscape_mountains_full.npy')
xmin, ymin, zmin = np.min(vertices, axis=0)
xmax, ymax, zmax = np.max(vertices, axis=0)

In [ ]:
# Convert png to numpy
heightmap_path = '../data/landscape_mountains_heightmap.png'
image = Image.open(heightmap_path)
heightmap = np.array(image)

In [ ]:
scale = (zmax - zmin) / (heightmap.max() - heightmap.min())
Z = scale * heightmap

In [ ]:
x = np.linspace(xmin, xmax, heightmap.shape[0])
y = np.linspace(ymin, ymax, heightmap.shape[1])
X, Y = np.meshgrid(x, y)

In [ ]:
player_start = np.array([-374.47859375, -723.12984375])
airsim_spiral_center = np.array([99, -449])
unreal_spiral_center = airsim_spiral_center + player_start
spiral_radius = 400

In [ ]:
i = (x > unreal_spiral_center[0] - spiral_radius) & (x < unreal_spiral_center[0] + spiral_radius)
j = (y > unreal_spiral_center[1] - spiral_radius) & (y < unreal_spiral_center[1] + spiral_radius)

In [ ]:
X = X[np.ix_(i,j)] - player_start[0]
Y = Y[np.ix_(i,j)] - player_start[1]
Z = Z[np.ix_(i,j)]

In [ ]:
# Downsample
# heightmap = heightmap[::5, ::5]
# X = X[::5, ::5]
# Y = Y[::5, ::5]

# Surface plot
plot_surface(x=X, y=Y, z=Z)

# Unreal Moon

In [ ]:
heightmap_path = '../data/unreal_moon_heightmap.png'
image = Image.open(heightmap_path)
heightmap = np.array(image)

In [ ]:
(np.array([5184.151855, -182487.484375, 1295.814209]) + np.array([110060.0, -148820.0, 25740.0])) / 200


In [ ]:
unreal_player_start_m = np.array([52, -1825])
center_idx = unreal_player_start_m + np.array([4064, 4064])

In [ ]:
# 2D heatmap plot
plt.imshow(heightmap, cmap='gray')

# draw a point at center idx
plt.scatter(center_idx[0], center_idx[1], color='red')

plt.show()

In [ ]:
unreal_player_start_m = np.array([52, -1825])
center_idx = unreal_player_start_m + np.array([4064, 4064])
print(f"center_idx: {center_idx}")

PATCH_WIDTH_M = 1000
patch_min = (center_idx[0]-PATCH_WIDTH_M//2, center_idx[1]-PATCH_WIDTH_M//2)
patch_max = (center_idx[0]+PATCH_WIDTH_M//2, center_idx[1]+PATCH_WIDTH_M//2)
print(f"patch_min: {patch_min}")
print(f"patch_max: {patch_max}")
patch = heightmap[patch_min[1]:patch_max[1], patch_min[0]:patch_max[0]]

# Plot the full heightmap with a box around the patch
plt.imshow(heightmap, cmap='gray')
plt.plot([patch_min[0], patch_max[0], patch_max[0], patch_min[0], patch_min[0]],
         [patch_min[1], patch_min[1], patch_max[1], patch_max[1], patch_min[1]], color='red')
plt.show()

In [ ]:
plt.imshow(patch, cmap='gray')

In [ ]:
x = np.linspace(-PATCH_WIDTH_M/2, PATCH_WIDTH_M/2, PATCH_WIDTH_M)
y = np.linspace(-PATCH_WIDTH_M/2, PATCH_WIDTH_M/2, PATCH_WIDTH_M)
X, Y = np.meshgrid(x, y)

# trajectory = np.load('../data/trajectory.npz')['trajectory']

HEIGHT_SCALE = 257

patch_adj = patch / HEIGHT_SCALE

starting_z = patch_adj[PATCH_WIDTH_M//2, PATCH_WIDTH_M//2]
# patch_adj = patch_adj - starting_z + trajectory[0,2]
patch_adj = patch_adj - starting_z 

# trajectory[:,2] -= trajectory[0,2]

# # rotate trajectory to align
# trajectory[:,[0,1]] = trajectory[:,[1,0]]
# trajectory[:,1] *= -1

# rotate patch to align
patch_adj = np.rot90(patch_adj)

In [ ]:
# query heights along trajectory
trajectory_heights = []
img_traj = []
for k in range(trajectory.shape[0]):
    x, y = trajectory[k,:2]
    i = int(-y) + PATCH_WIDTH_M//2
    j = int(x) + PATCH_WIDTH_M//2
    trajectory_heights.append(patch_adj[i, j])
    img_traj.append((i, j))

trajectory_heights = np.array(trajectory_heights)

# Compute error
error = trajectory_heights - trajectory[:,2]
print(f"Mean error: {np.mean(np.abs(error))}")

# Plot trajectory
plt.imshow(patch_adj, cmap='gray')
plt.plot([j for i,j in img_traj], [i for i,j in img_traj], color='red')
plt.show()

In [ ]:
fig = go.Figure()
# plot_path_3d(trajectory[:,0], trajectory[:,1], trajectory[:,2], fig=fig)
plot_surface(x=X, y=-Y, z=patch_adj, fig=fig)

In [ ]:
# TODO: plot camera poses
import json

transforms_path = '../../nerfstudio/data/moon_spiral_2/transforms.json'
with open(transforms_path, 'r') as f:
    transforms = json.load(f)

poses = []
for frame in transforms['frames']:
    T = np.array(frame['transform_matrix'])
    R, t = T[:3, :3], 5.0 * T[:3, 3]
    poses.append((R, t))

poses = pose_traces(poses)

# Create and show the plot with all the traces
# fig = go.Figure(data=poses)
fig.add_traces(poses)
fig.update_layout(height=900, width=1600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
fig.write_html('../data/moon_trajectory.html')

In [ ]:
from nemo.util import generate_ply_from_dem

dem_points = np.stack([X.flatten(), -Y.flatten(), patch_adj.flatten()], axis=1)
colors = 128 * np.ones_like(dem_points)

generate_ply_from_dem(dem_points, colors, '../data/sparse_pc.ply')

# Lunar DEM

In [ ]:
img_np = np.load('../data/lunar_dem.npy')

In [ ]:
img_np.shape

In [ ]:
img_np_ds = img_np[::100, ::100]
# make figure biffer
fig = plt.figure(figsize=(15, 30))
plt.imshow(img_np_ds)
plt.show()

In [ ]:
# Look at 1000x1000 patch in the top left corner
patch = img_np[:1000, :1000]
Z = patch

# Display the image
plt.imshow(patch)
plt.show()

### Surface plot to scale

In [ ]:
xs = torch.linspace(0, 118500, steps=Z.shape[0], device=device)
ys = torch.linspace(0, 118400, steps=Z.shape[1], device=device)
x, y = torch.meshgrid(xs, ys, indexing='xy')

fig = go.Figure(data=[go.Surface(z=Z, x=x.cpu().numpy(), y=y.cpu().numpy())])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

### Scale to -1 to 1

In [ ]:
Z_normalized = 1 * (Z - np.min(Z)) / (np.max(Z) - np.min(Z))   # 0 to 1
xs = torch.linspace(-1, 1, steps=Z.shape[0], device=device)
ys = torch.linspace(-1, 1, steps=Z.shape[1], device=device)
x, y = torch.meshgrid(xs, ys, indexing='xy')
xy = torch.hstack((x.reshape(-1, 1), y.reshape(-1, 1)))

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z_normalized, x=x.cpu().numpy(), y=y.cpu().numpy())])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
siren = Siren(in_features=2, 
              out_features=1, 
              hidden_features=256,
              hidden_layers=3, 
              outermost_linear=True,
              first_omega_0=30.0,
              hidden_omega_0=100.0).to(device)

siren.load_state_dict(torch.load('../models/lunar_dem_siren.pth'))
siren.eval()
pass

In [ ]:
with torch.no_grad():
    pred, coords = siren(xy)

In [ ]:
# Plot the predictions
fig = go.Figure(data=[go.Surface(z=pred.cpu().numpy().reshape(Z.shape), x=x.cpu().numpy(), y=y.cpu().numpy())])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

# SIREN

### Mt Bruno elevation data

In [ ]:
# import pandas as pd
# z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
# z_data.values

# np.save('../data/mt_bruno_elevation.npy', z_data.values)

In [ ]:
z_vals = np.load('../data/mt_bruno_elevation.npy')

# Guess the xy scale
xy = 200 * np.mgrid[-12:13, -12:13]
xvals = xy[0]
yvals = xy[1]

# Plot the data
fig = go.Figure(data=[go.Surface(z=z_vals, x=xvals, y=yvals)])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
# Scale the XY data to -1 to 1
xy_scaled = xy / 2400
x_scaled = xy_scaled[0]
y_scaled = xy_scaled[1]
z_scaled = z_vals / np.max(z_vals)

In [ ]:
# Plot the scaled data
fig = go.Figure(data=[go.Surface(z=z_scaled, x=x_scaled, y=y_scaled)])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
# Fit a Siren network to the data

siren = Siren(in_features=2, out_features=1, hidden_features=256,
                hidden_layers=3, outermost_linear=True).to(device)

In [ ]:
# Train the network

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(siren.parameters(), lr=1e-5)

# Convert the data to torch tensors
xy_tensor = torch.tensor(xy_scaled, dtype=torch.float32).to(device)
xy_tensor = xy_tensor.reshape(2, -1).T
z_tensor = torch.tensor(z_scaled, dtype=torch.float32).to(device)
z_tensor = z_tensor.reshape(-1, 1)

# Train the network
for step in range(5000):
    # Forward pass
    # pred, coords = siren(xy_tensor)
    pred = siren(xy_tensor)

    # Compute loss
    loss = criterion(pred, z_tensor)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 500 steps
    if step % 500 == 0:
        print(f"Step {step}, Loss {loss.item()}")

In [ ]:
torch.save(siren.state_dict(), '../models/mt_bruno_siren.pt')

In [ ]:
# Sample the Siren network to get the predicted elevation
with torch.no_grad():
    pred, coords = siren(xy_tensor)

# Plot the predictions
fig = go.Figure(data=[go.Surface(z=pred.cpu().numpy().reshape(25, 25), x=x_scaled, y=y_scaled)])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
# MSE loss
criterion(pred, z_tensor)

Increase the sampling resolution

In [ ]:
xs = torch.linspace(-1, 1, steps=100, device=device)
ys = torch.linspace(-1, 1, steps=100, device=device)
x, y = torch.meshgrid(xs, ys, indexing='xy')
xy = torch.hstack((x.reshape(-1, 1), y.reshape(-1, 1)))

In [ ]:
with torch.no_grad():
    pred, coords = siren(xy)

# Plot the predictions
fig = go.Figure(data=[go.Surface(z=pred.cpu().numpy().reshape(100, 100), x=x.cpu().numpy(), y=y.cpu().numpy())])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

# Plan a path over the SIREN

Compute spatial derivatives

In [ ]:
siren = Siren(in_features=2, out_features=1, hidden_features=256,
                hidden_layers=3, outermost_linear=True).to(device)

siren.load_state_dict(torch.load('../models/mt_bruno_siren.pt'))
siren.eval()

In [ ]:
z_pred, coords = siren(xy)

In [ ]:
def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad

In [ ]:
z_xy_grad = gradient(z_pred, coords)

x_grad = z_xy_grad[:, 0].detach().cpu().numpy().reshape(100, 100)
y_grad = z_xy_grad[:, 1].detach().cpu().numpy().reshape(100, 100)

In [ ]:
# Visualize the gradients at 2D heatmaps

from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=('X Gradient', 'Y Gradient'))
fig.add_trace(go.Heatmap(z=x_grad), row=1, col=1)
fig.add_trace(go.Heatmap(z=y_grad), row=1, col=2)
fig.update_layout(width=1400, height=700, scene_aspectmode='data')
fig.show()

## Plan a discrete path

from $(-1, 1)$ to $(1, 1)$

We want the path that minimizes distance (number of waypoints) and XY gradient values

In [ ]:
GRID_LEN = 100

# start_idx = (0, 0)                   # (-1, -1)
# end_idx = (GRID_LEN-1, GRID_LEN-1)   # (1, 1)

start_idx = (0, GRID_LEN-1)                   # (-1, -1)
end_idx = (GRID_LEN-1, 0)   # (1, 1)

grad_costmat = (np.abs(x_grad) + np.abs(y_grad))

In [ ]:
# Run A* on the cost matrix
from global_planner import GlobalPlanner

gp = GlobalPlanner(grad_costmat)
path = gp.plan(start_idx, end_idx)

fig, ax = plt.subplots(figsize=(10,10))
gp.plot(ax)
plt.show()

In [ ]:
grad_costmat

In [ ]:
path_xs = xs[path[:,0]]
path_ys = ys[path[:,1]]
path_xy = torch.hstack((path_xs[:,None], path_ys[:,None]))
path_zs, _ = siren(path_xy)

In [ ]:
# Plot path on surface plot
fig = go.Figure()
fig.add_trace(go.Surface(z=pred.cpu().numpy().reshape(100, 100), x=x.cpu().numpy(), y=y.cpu().numpy()))
fig.add_trace(go.Scatter3d(x=path_xs.detach().cpu().numpy(), 
                           y=path_ys.detach().cpu().numpy(), 
                           z=path_zs.detach().cpu().numpy().flatten(), 
                           mode='markers', marker=dict(size=3, color='red')))
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
# save fig as html
fig.write_html('path.html')

# Point Cloud DEM

Form k-D tree over 3D point cloud.

For 2D point $(x,y)$, find $N$ points in $xy$ (or all points within some $xy$ radius).

Fit a plane to these points using RANSAC to obtain slope



In [ ]:
# Load COLMAP point cloud

In [ ]:
tile_name = 'SLDEM2015_256_60S_0S_120_240'
img_np = np.load(f'../data/{tile_name}.npy')

In [ ]:
N = 250
patch = img_np[:N, :N]
z = patch

# Scale x and y
xs = np.linspace(0, 118.45*N, num=z.shape[0])
ys = np.linspace(0, 118.45*N, num=z.shape[1])
x, y = np.meshgrid(xs, ys, indexing='xy')

pc_dem = np.vstack((x.flatten(), y.flatten(), z.flatten())).T

# Plot the data
fig = plot_3d_points(x=pc_dem[:,0], y=pc_dem[:,1], z=pc_dem[:,2], color=pc_dem[:,2], markersize=1)
fig.show()

In [ ]:
from scipy.spatial import kdtree

pc_tree = kdtree(pc_dem)

In [ ]:
query = np.array([0, 0])



# Mesh DEM

Triangulate 3D points using Delaunay.

Move between triangles (triangles are nodes), shared faces are edges. Slope is slope of triangle.
* Not dynamically feasible

For 2D point $(x,y)$, find the triangle which intersects vertical line extending up from $(x,y)$. Slope is slope of this triangle


In [ ]:
from scipy.spatial import Delaunay

In [ ]:
mesh = Delaunay(pc_dem)

In [ ]:
mesh.vertices

In [ ]:
mesh

# Implicit Surface DEM

Surface represented with SDF

Plan path in 3D, but with constraint loss to stay on the surface 